In [1]:
from solartwins import *
from scipy import optimize
from tqdm import tqdm

#np.seterr(divide='ignore', invalid='ignore')

In [2]:
t

star_name,age,age_err,Fe,Fe_err,O,O_err,Na,Na_err,Mg,Mg_err,Al,Al_err,Si,Si_err,S,S_err,Ca,Ca_err,V,V_err,Mn,Mn_err,Co,Co_err,Ni,Ni_err,Cu,Cu_err,Zn,Zn_err,Sr,Sr_err,Y,Y_err,Zr,Zr_err,Ba,Ba_err,La,La_err,Ce,Ce_err,Pr,Pr_err,Nd,Nd_err,Sm,Sm_err,Eu,Eu_err,Gd,Gd_err,Dy,Dy_err,C,C_err,Ti,Ti_err,Sc,Sc_err,Cr,Cr_err
bytes9,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
HIP10175,3.2,0.4,0.0,0.002,0.024,0.009,-0.051000000000000004,0.022,-0.006000000000000002,0.007,-0.034,0.008,-0.009999999999999998,0.003,0.0020000000000000018,0.012,0.023,0.005,-0.0019999999999999983,0.004,-0.029,0.005,-0.028,0.004,-0.04100000000000001,0.003,-0.05600000000000001,0.013,-0.048,0.006,0.081,0.005,0.066,0.007,0.087,0.012,0.11599999999999999,0.005,0.118,0.029,0.104,0.016,0.16999999999999998,0.008,0.127,0.01,0.057,0.013,0.089,0.005,0.08,0.005,0.065,0.005,-0.05176923076923076,0.013000000000000001,0.012,0.01,-0.019000000000000003,0.020999999999999998,0.007750000000000001,0.012
HIP101905,1.2,0.3,0.0,0.004,-0.06899999999999999,0.014,-0.093,0.012,-0.025999999999999995,0.008,-0.033999999999999996,0.002,-0.024999999999999994,0.004,-0.104,0.027,0.024000000000000007,0.005,0.0050000000000000044,0.009,-0.04599999999999999,0.006,-0.03799999999999999,0.005,-0.03799999999999999,0.005,-0.08499999999999999,0.031,-0.09699999999999999,0.016,0.097,0.006,0.08100000000000002,0.011,0.08499999999999999,0.01,0.14,0.01,0.10200000000000001,0.031,0.119,0.012,0.094,0.014,0.131,0.01,0.05399999999999999,0.021,0.07600000000000001,0.014,0.049000000000000016,0.006,0.066,0.066,-0.11374193548387096,0.031,0.009076923076923078,0.013000000000000001,-0.008058823529411757,0.017,-0.004749999999999994,0.016
HIP102040,2.4,0.4,0.0,0.003,0.022,0.006,-0.071,0.008,0.0020000000000000018,0.003,-0.021000000000000005,0.008,-0.0040000000000000036,0.005,-0.06499999999999999,0.014,0.034,0.005,0.0,0.006,-0.06299999999999999,0.005,-0.012999999999999998,0.007,-0.03799999999999999,0.004,-0.06599999999999999,0.034,-0.077,0.02,0.062,0.005,0.054000000000000006,0.007,0.081,0.011,0.128,0.012,0.118,0.018,0.14900000000000002,0.016,0.158,0.01,0.165,0.011,0.113,0.023,0.14600000000000002,0.017,0.094,0.006,0.108,0.028,-0.019789473684210527,0.056999999999999995,0.029545454545454552,0.011,0.0178,0.034999999999999996,-0.007249999999999993,0.016
HIP102152,8.6,0.3,0.0,0.003,0.051000000000000004,0.01,-0.027999999999999997,0.014,0.031,0.01,0.026000000000000002,0.004,0.003000000000000001,0.003,-0.012,0.022,0.006,0.005,-0.009000000000000001,0.006,-0.026999999999999996,0.005,-0.018000000000000002,0.008,-0.017,0.004,0.002,0.004,0.004,0.011,-0.11,0.005,-0.104,0.007,-0.094,0.012,-0.049,0.009,-0.062,0.019,-0.023,0.013,0.0,0.017,0.0010000000000000009,0.009,0.015,0.01,0.023,0.019,0.028,0.005,-0.031,0.027,0.001285714285714285,0.014,0.0035454545454545456,0.011,-0.00975,0.016,-0.002230769230769232,0.013
HIP10303,5.8,0.4,0.0,0.003,-0.022999999999999993,0.007,-0.0020000000000000018,0.012,0.017,0.005,0.033000000000000015,0.004,0.0040000000000000036,0.002,-0.023999999999999994,0.014,0.0050000000000000044,0.003,0.015,0.006,0.019000000000000003,0.004,0.0,0.005,0.020000000000000004,0.004,0.013999999999999999,0.021,0.0040000000000000036,0.02,0.032000000000000015,0.005,0.019000000000000003,0.006,0.0040000000000000036,0.014,-0.01999999999999999,0.008,-0.031,0.006,-0.027999999999999997,0.015,0.008000000000000007,0.012,-0.013999999999999999,0.007,-0.010999999999999996,0.016,0.01100000000000001,0.013,0.030000000000000013,0.005,-0.0049999999999999906,0.021,-0.033571428571428565,0.007,0.009727272727272737,0.011,0.005250

In [3]:
deltas = []
deltas_no_temp= []
scatter = []
scatter_no_temp =[]

star_elements = []
for n in t.colnames:
    if len(n) < 3 :
        star_elements.append(n) #list of elements in table
    elements = np.array(star_elements)
    
for r, star in enumerate(elements):
    #tbl = t[r] #inputted star's row 
    x0 = (.01, .03, .7)
    
    #con_temp = []
    #for n in elements: 
        #con_temp.append(tc_map[n])
    #temps = np.copy(con_temp)
    #random.shuffle(temps)
    
    #star_abundance = []
    #abundance_error = []
    #for n in elements:
        #star_abundance.append([t[n]])
        #abundance_error.append([t[n + '_err']])
    
    #abundance_notemp = residuals(temps, star_abundance, abundance_error)
    resid =  residuals(t['age'], t[star], t[star+ '_err'])
    
    #BEFORE REMOVING TEMP TRENDS
    
    #delt_temp = scipy.optimize.minimize(twodnlnL, x0, args = (t['age'], t[star], t['age_err'], t[star + '_err']))
    delt_temp = scipy.optimize.minimize(nlnL, x0, args = (t['age'], t[star], t[star + '_err']))
    deltavlue = delt_temp['x'][2]
    deltas.append(deltavlue)
    
    #delta = scipy.optimize.minimize(nlnL, x0, args = (temps, star_abundance, abundance_error))
    #deltanotempvlue = delta['x'][2]
    #scatter.append(deltanotempvlue)
    
    #AFTER REMOVING TEMP
    #delt_notemp = scipy.optimize.minimize(twodnlnL, x0, args = (t['age'], resid, t['age_err'], t[star + '_err']))
    delt_notemp = scipy.optimize.minimize(nlnL, x0, args = (t['age'], resid, t[star + '_err']))

    value = delt_notemp['x'][2]
    deltas_no_temp.append(value)
        
    #delta_notemp = scipy.optimize.minimize(nlnL, x0, args = (temps, abundance_notemp, abundance_error))
    #deltanot = delta_notemp['x'][2]
    #scatter_no_temp.append(deltanot)

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 15
fig_size[1] = 12
plt.rcParams["figure.figsize"] = fig_size
plt.scatter(stardeltas, star_notemp - stardeltas)
plt.scatter(deltanotemp, deltanotemp - stardeltas)

plt.ylim(-.0005, .0005)

In [4]:
def residuals(x, y, error):
    mborig = find_m_b(x, y, error)
    m = mborig[0]
    b = mborig[1]

    predicted_values = [] #y values from slope    
    pv = 0
    for u in x:
        pv = (m*u) + b
        predicted_values.append(pv)
        pv = 0

    prev = np.array(predicted_values)
    abu = np.array(y)
    diff = abu - prev #difference between slope and measured values 
    return diff

In [34]:
element_residuals = []
std_residuals = []
std_abundances = []
percent_diff_abund =[]
percent_diff_resid = []

for u, ment in enumerate(star_elements):
    std_abundances.append(np.std(t[ment]))
    element_residuals.append(residuals(t['age'], t[ment], t[ment + '_err']))
    std_residuals.append(np.std(element_residuals[u]))

standard_dev_table = Table([std_abundances, deltas, std_residuals, deltas_no_temp], names=('StDev Abund', 
                                                                'Delta Abund','StDevResid', 'Delta Resid'))            

In [6]:
standard_dev_table

StDev Abund,Delta Abund,StDevResid,Delta Resid
float64,float64,float64,float64
0.0,-7.450473707648826e-09,0.0,-7.450473707648826e-09
0.033611419319191496,-0.024338345544178824,0.026719476522191497,-0.02433834461856655
0.03595281172982548,-0.02742068460389971,0.029209208399030927,0.0274206702049749
0.029965966133224834,-0.012354493810729915,0.016754876051260637,-0.012354491322156573
0.038449476465674134,-0.016672949829174884,0.018223317485673792,-0.01667294563804212
0.01925125259934009,-0.011349978440814774,0.011891489271426116,-0.01134997487116162
0.038543609867158206,-0.02477621693457403,0.032257028387927114,-0.024776224521264663
0.010348971498301865,-0.008918270112154986,0.010006651266438777,-0.008918269883639402
0.012400154871228416,-0.009216966881982475,0.012162507827491653,-0.009216975049163956


In [7]:
def delta(param, x, y): # Hogg 2010 eqn 30 : ∆i = (vˆT Zi) − b cos θ                                                                                                                                        
    (m,b,d) = param
    theta = np.arctan(m)
    v = np.array([- np.sin(theta), np.cos(theta)]) # v = [− sin θ  cos θ]                                                                                                                                   

    disp = np.zeros_like(y)
    for i, (ys, xs) in enumerate(zip(y, x)):
        z0 = np.asarray([0.0, b])
        zi = np.asarray([xs, ys])
        disp[i] = np.dot( v, zi - z0 )
    return disp


In [8]:
#2D uncertainties: lnL = K -  sum [delta^2 / 2* sum^2]    
def twodnlnL(param, x, y, errx, erry):
    K = 0
    delt = delta(param, x, y)
    (m,b,d) = param

    #Σ^2 = vˆT Si v   
    theta = np.arctan(m)
    v = np.array([- np.sin(theta), np.cos(theta)]) # v = [− sin θ,  cos θ] 

    var = np.zeros_like(erry)
    for i, (dy, dx) in enumerate(zip(erry, errx)):
        cov = np.eye(2)
        cov[0,0] = dx**2
        cov[1,1] = dy**2
        var[i] = np.dot(v.T, np.dot(cov, v))

    sigmasq = var + np.exp(2.*d)
    return 0.5 * np.sum(delt**2/sigmasq + np.log(sigmasq))

In [22]:
def stdevlnL(param, x, y, error): # Hogg: lnL = -1/2 sum [(y- (mx+b))^2/(err^2 + d^2)] -1/2 sum ln(err^2 + d^2)   
    x = np.array(x)
    y = np.array(y)
    error = np.array(error)
    (m,b,s) = param
    
    sum1 = 0
    for j, val in enumerate(x):
        mxb = m*val + b
        value = ((y[j] - mxb)**2)/((error[j]**2) + (s**2))
        sum1 = sum1 + value
    sum1 = sum1 *(-1/2)

    num = 0
    for e, ooh in enumerate(error):
        val2 = np.log((ooh**2) + (s**2))
        num = num + val2
    num = num * (-1/2)

    sdlnL = sum1 + num
    return sdlnL

def stdevnlnL(param, x, y, error):
    nlnL= lnL(param, x,y, error)
    return - nlnL

In [24]:
def stdev(x):
    #(m,b) = param 
    mean = np.mean(x)
    fun = 0
    
    for c in x: 
        j = (c-mean)**2
        fun = fun + j 
    
    s = np.sqrt(fun/(len(x)))
    return s

In [26]:
sd = stdev(t['Mg'])
x0 = (0.009, -0.03, sd)
res = scipy.optimize.minimize(stdevnlnL, x0, args=(t['age'], t['Mg'], t['Mg_err']))
res

      fun: -249.39633173623352
 hess_inv: array([[ 7.02653762e-07, -3.61207279e-06,  1.78609643e-07],
       [-3.61207279e-06,  2.19753113e-05, -9.37622627e-07],
       [ 1.78609643e-07, -9.37622627e-07,  2.58680793e-06]])
      jac: array([0.00693321, 0.00014305, 0.00094032])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 347
      nit: 12
     njev: 67
   status: 2
  success: False
        x: array([ 0.00984497, -0.03665834,  0.01235448])

In [13]:
x1 = (9, 4)
standard = []
for g in element_residuals:
    standard.append(stdev(x1, g))

In [20]:
guess = scipy.optimize.minimize(stdev, (1,3), t['age'])
guess

      fun: 2.3941190020230305
 hess_inv: array([[1, 0],
       [0, 1]])
      jac: array([0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 4
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([1., 3.])

In [31]:
opstdev = []
stdev_abund = []
for c in elements:
    sd = stdev(t[c])
    stdev_abund.append(sd)
    x0 = (0.009, -0.03, sd)
    res = scipy.optimize.minimize(stdevnlnL, x0, args=(t['age'], t[c], t[c + '_err']))
    opstdev.append(res['x'][2])

In [41]:
Table([opstdev, deltas, stdev_abund, std_abundances], names = ('Optimized StDev', 'Deltas', 'StDev', 'np StDev'))

Optimized StDev,Deltas,StDev,np StDev
float64,float64,float64,float64
3.126878073013827e-08,-7.450473707648826e-09,0.0,0.0
0.024338330921975588,-0.024338345544178824,0.033611419319191496,0.033611419319191496
0.027420669439350585,-0.02742068460389971,0.03595281172982548,0.03595281172982548
0.012354481965710833,-0.012354493810729915,0.02996596613322483,0.029965966133224834
0.01667293471648078,-0.016672949829174884,0.038449476465674134,0.038449476465674134
0.011349962840417803,-0.011349978440814774,0.019251252599340084,0.01925125259934009
0.024776202007551568,-0.02477621693457403,0.03854360986715821,0.038543609867158206
0.008918256329133104,-0.008918270112154986,0.010348971498301865,0.010348971498301865
0.009216958192162325,-0.009216966881982475,0.012400154871228418,0.012400154871228416
